# Kelompok:
#### Peter Abednego Wijaya 			235150300111013
#### Gilang Shido Faizalhaq			235150300111011
#### Rizqy Jauhary Atsaany			235150300111038

## Pastikan install library terlebih dahulu. Run di terminal:
### pip install pandas requests openpyxl

## Langkah 1, cek data-data yang disediakan pada weather API 

In [2]:
import requests
import json # Menggunakan json

# --- KONFIGURASI ---
# API key
API_KEY = "242819b7dfe84b4488a34756251809" 
LOKASI_CONTOH = "Malang" # Coba ambil data untuk Malang
# --- AKHIR KONFIGURASI --

# URL endpoint dari WeatherAPI
url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={LOKASI_CONTOH}&aqi=no"

print(f"Mengambil data cuaca untuk: {LOKASI_CONTOH}...")

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Error jika status code bukan 2xx
    
    data = response.json()
    
    # Cetak seluruh response JSON dengan format yang rapi
    print("\n--- Struktur Data JSON Lengkap ---")
    print(json.dumps(data, indent=4))
    
except requests.exceptions.RequestException as e:
    print(f"Gagal mendapatkan data: {e}")

Mengambil data cuaca untuk: Malang...

--- Struktur Data JSON Lengkap ---
{
    "location": {
        "name": "Malang",
        "region": "East Java",
        "country": "Indonesia",
        "lat": -7.9797,
        "lon": 112.6304,
        "tz_id": "Asia/Jakarta",
        "localtime_epoch": 1758774178,
        "localtime": "2025-09-25 11:22"
    },
    "current": {
        "last_updated_epoch": 1758773700,
        "last_updated": "2025-09-25 11:15",
        "temp_c": 33.1,
        "temp_f": 91.6,
        "is_day": 1,
        "condition": {
            "text": "Patchy rain nearby",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/176.png",
            "code": 1063
        },
        "wind_mph": 2.2,
        "wind_kph": 3.6,
        "wind_degree": 316,
        "wind_dir": "NW",
        "pressure_mb": 1010.0,
        "pressure_in": 29.83,
        "precip_mm": 0.07,
        "precip_in": 0.0,
        "humidity": 56,
        "cloud": 25,
        "feelslike_c": 35.8,
        "feels

# 2. Membuat file excel bernama "DaftarKecamatan" dan pada kolom pertama di isi kecamatan-kecamatan di Jawa Timur

# 3. Membuat script untuk membaca file Excel, mengambil data cuaca untuk setiap kecamatan menggunakan multi-threading, dan menuliskannya kembali ke file Excel yang sama.

In [1]:
# --- 1. IMPORT LIBRARY YANG DIBUTUHKAN ---
import pandas as pd
import requests
import threading
from datetime import datetime

# --- 2. KONFIGURASI UTAMA ---
API_KEY = "242819b7dfe84b4488a34756251809"
NAMA_FILE_EXCEL = "DaftarKecamatan_kelompok_6.xlsx"
NAMA_KOLOM_KECAMATAN = "Kecamatan"

# =========== BAGIAN UNTUK METODE MULTI-THREADING =============

# --- 3A. PERSIAPAN UNTUK MULTI-THREADING ---
results = []
lock = threading.Lock()

# --- 4A. FUNGSI GET DATA (UNTUK THREAD) ---
def get_weather_data_threaded(kecamatan, index):
    """Fungsi ini dijalankan oleh setiap thread. Hasilnya disimpan di list global 'results'."""
    url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={kecamatan}&aqi=no"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        weather_info = {
            'Last Update': data['location']['localtime'],
            'Suhu (°C)': data['current']['temp_c'],
            'Kelembapan (%)': data['current']['humidity'],
            'Kondisi Cuaca': data['current']['condition']['text'],
            'Kecepatan Angin (km/h)': data['current']['wind_kph'],
            'Arah Angin': data['current']['wind_dir'],
            'Sinar UV': data['current']['uv'],
            'Cloud ': data['current']['cloud']
        }
        print(f"✅ [Thread] Berhasil -> {kecamatan}")
    except requests.exceptions.RequestException:
        print(f"❌ [Thread] Gagal    -> {kecamatan}")
        weather_info = {
            'Last Update': 'Error', 'Suhu (°C)': 'Error', 'Kelembapan (%)': 'Error',
            'Kondisi Cuaca': 'Error', 'Kecepatan Angin (km/h)': 'Error',
            'Arah Angin' : 'Error', 'Sinar UV': 'Error'
        }
    with lock:
        results.append({'index': index, 'data': weather_info})

# --- 5A. FUNGSI UTAMA (MULTI-THREAD) ---
def run_w_multithread():
    start_time = datetime.now()
    df = pd.read_excel(NAMA_FILE_EXCEL, engine='openpyxl')
    threads = []
    
    # Kosongkan list results setiap kali fungsi dijalankan
    results.clear()
    
    print(f"🚀 Memulai pengambilan data untuk {len(df)} kecamatan (Multi-Thread)...")
    for index, row in df.iterrows():
        kecamatan = row[NAMA_KOLOM_KECAMATAN]
        if pd.notna(kecamatan):
            thread = threading.Thread(target=get_weather_data_threaded, args=(kecamatan, index))
            threads.append(thread)
            thread.start()

    for thread in threads:
        thread.join()

    print("\nSemua thread selesai. Menulis data ke Excel...")
    for result in results:
        idx = result['index']
        for key, value in result['data'].items():
            df.loc[idx, key] = value
            
    df.to_excel(NAMA_FILE_EXCEL, index=False)
    end_time = datetime.now()
    print(f"\n✅ Selesai! Data cuaca telah ditulis ke '{NAMA_FILE_EXCEL}'.")
    print(f"Total waktu eksekusi (Multi-Thread): {end_time - start_time}")

# =========== BAGIAN UNTUK METODE SEQUENTIAL ============

# --- 4B. FUNGSI GET DATA (UNTUK SEQUENTIAL) ---
def get_weather_data_sequential(kecamatan):
    """Fungsi ini dijalankan satu per satu. Hasilnya langsung di-'return'."""
    url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={kecamatan}&aqi=no"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        weather_info = {
            'Last Update': data['location']['localtime'],
            'Suhu (°C)': data['current']['temp_c'],
            'Kelembapan (%)': data['current']['humidity'],
            'Kondisi Cuaca': data['current']['condition']['text'],
            'Kecepatan Angin (km/h)': data['current']['wind_kph'],
            'Arah Angin': data['current']['wind_dir'],
            'Sinar UV': data['current']['uv']
        }
        print(f"✅ [Seq] Berhasil -> {kecamatan}")
    except requests.exceptions.RequestException:
        print(f"❌ [Seq] Gagal    -> {kecamatan}")
        weather_info = {
            'Last Update': 'Error', 'Suhu (°C)': 'Error', 'Kelembapan (%)': 'Error',
            'Kondisi Cuaca': 'Error', 'Kecepatan Angin (km/h)': 'Error', 
            'Arah Angin' : 'Error', 'Sinar UV': 'Error'
        }
    return weather_info

# --- 5B. FUNGSI UTAMA (SEQUENTIAL) ---
def run_sequentially():
    start_time = datetime.now()
    df = pd.read_excel(NAMA_FILE_EXCEL, engine='openpyxl')
    
    print(f"🐢 Memulai pengambilan data untuk {len(df)} kecamatan (Sekuensial)...")
    for index, row in df.iterrows():
        kecamatan = row[NAMA_KOLOM_KECAMATAN]
        if pd.notna(kecamatan):
            weather_data = get_weather_data_sequential(kecamatan)
            for key, value in weather_data.items():
                df.loc[index, key] = value
            
    print("\nSemua data telah diambil. Menulis data ke Excel...")
    df.to_excel(NAMA_FILE_EXCEL, index=False)
    end_time = datetime.now()
    print(f"\n✅ Selesai! Data cuaca telah ditulis ke '{NAMA_FILE_EXCEL}'.")
    print(f"Total waktu eksekusi (Sekuensial): {end_time - start_time}")


# ======================== PANEL KONTROL EKSEKUSI ===========================
if __name__ == "__main__":
    # --- Pilihan 1: Mode Cepat (Multi-Thread) ---
     run_w_multithread()
    
    # --- Pilihan 2: Mode Sabar (Sekuensial) ---
    #run_sequentially()

🚀 Memulai pengambilan data untuk 591 kecamatan (Multi-Thread)...
✅ [Thread] Berhasil -> Sedati, Sidoarjo
✅ [Thread] Berhasil -> Kenjeran, Surabaya
✅ [Thread] Berhasil -> Kauman, Ponorogo
✅ [Thread] Berhasil -> Bungkal, Ponorogo
✅ [Thread] Berhasil -> Sukorejo, Ponorogo
✅ [Thread] Berhasil -> Sambit, Ponorogo
✅ [Thread] Berhasil -> Suko Manunggal, Surabaya
✅ [Thread] Berhasil -> Asemrowo, Surabaya
✅ [Thread] Berhasil -> Bandar, Pacitan
✅ [Thread] Berhasil -> Tenggilis Mejoyo, Surabaya
✅ [Thread] Berhasil -> Wonocolo, Surabaya
✅ [Thread] Berhasil -> Pabean Cantian, Surabaya
✅ [Thread] Berhasil -> Porong, Sidoarjo
✅ [Thread] Berhasil -> Ngrayun, Ponorogo
✅ [Thread] Berhasil -> Bubutan, Surabaya
✅ [Thread] Berhasil -> Jabon, Sidoarjo
✅ [Thread] Berhasil -> Benowo, Surabaya
✅ [Thread] Berhasil -> Gayungan, Surabaya
✅ [Thread] Berhasil -> Pudak, Ponorogo
✅ [Thread] Berhasil -> Pakal, Surabaya
✅ [Thread] Berhasil -> Wonokromo, Surabaya
✅ [Thread] Berhasil -> Sukolilo, Surabaya
✅ [Thread] Berh